In [1]:
from os import path,listdir
from datetime import datetime,timedelta
import time
import re
import pandas as pd
import numpy as np
import random
import pytz
from tqdm import tqdm
import bz2
import json
import glob
local_tz = 'UTC'

In [2]:
chars0=['ó','é','í','ş','ã','İ','ğ','ç','ü','É','â','Ç','õ','ł','ą','Ś','ø','ń','ț','å','Å','ß', 'æ', 'Ž','ş', 'ə','Ö','ı','á','î','ñ','ö','ź','ú','è','Ł','ę','Ş','ä','ë','ô','ș','ū','č','Š','Þ','ė','Ä','ă','ì','š','i','ć','ň','ž','ư','ơ','ê','à','ð','ő','Ü','ý','ď','Á','ř','Č','Ú']
chars1=['o','e','i','s','a','I','g','c','u','E','a','C','o','l','a','s','o','n','t','a','A','ss','ae','Z','sh','a','O','i','a','i','n','o','z','u','e','L','e','S','a','e','o','s','u','c','S','P','e','A','a','i','s','i','c','n','z','u','o','e','a','d','o','U','y','d','A','r','C','U']
dicUnicode2En=dict(zip(chars0, chars1))

In [3]:
df_countries=pd.read_csv('data/countries.csv', index_col=None)
df_countries['Name']=df_countries['Name'].str.lower()
df_countries.columns=['country','countryCode']

df_bf=pd.read_csv('data/bf/bf_matches.csv', index_col=None)
df_bf=df_bf[~df_bf['halfTime'].isna()]
df_bf=df_bf[df_bf['halfTime']!='0']
df_bf['inplayTime']=pd.to_datetime(df_bf['inplayTime'])
df_bf['home_name_low']=df_bf['home_name'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_bf['away_name_low']=df_bf['away_name'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()

df_ss=pd.read_csv('data/matches_done.csv', index_col=None)
df_ss['ts']=pd.to_datetime(df_ss['ts'])
df_ss=df_ss.merge(df_countries, on='country', how='left')
df_ss.loc[df_ss['country']=='england','countryCode']='GB'
df_ss.loc[df_ss['country']=='scotland','countryCode']='GB'
df_ss.loc[df_ss['country']=='czech-republic','countryCode']='CZ'
df_ss.loc[df_ss['country']=='russia','countryCode']='RU'
df_ss.loc[df_ss['country']=='usa','countryCode']='US'
df_ss.loc[df_ss['homeTeamShort']=='???','homeTeamShort']='AEK Athens'
df_ss.loc[df_ss['awayTeamShort']=='???','awayTeamShort']='AEK Athens'
df_ss['homeTeamShortLow']=df_ss['homeTeamShort'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_ss['awayTeamShortLow']=df_ss['awayTeamShort'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()

#df_ss1=pd.read_csv('data/matches2.csv', index_col=None)
#pd.concat([df_ss,df_ss1], axis=0).to_csv('data/matches_done2.csv', index=False)

In [4]:
def slice_df(df, cc, str1, str2, type='00'):
    if type=='00':
        return df.loc[(df['countryCode']==cc) & (df['home_name_low']==str1) & (df['away_name_low']==str2)]
    elif type=='10':
        return df.loc[(df['countryCode']==cc) & (df['home_name_low'].str.contains(str1)) & (df['away_name_low']==str2)]
    elif type=='01':
        return df.loc[(df['countryCode']==cc) & (df['home_name_low']==str1) & (df['away_name_low'].str.contains(str2))]
    else:
        return df.loc[(df['countryCode']==cc) & (df['home_name_low'].str.contains(str1)) & (df['away_name_low'].str.contains(str2))]

In [5]:
binds=[]
d= datetime(2015, 5, 1)
end_date= datetime(2020, 11, 1)
while d<end_date:
    print(d)
    dloc=pytz.timezone(local_tz).localize(d)
    df_ss_day=df_ss[(df_ss['ts']>=dloc) & (df_ss['ts']<dloc+timedelta(days=1))]
    df_bf_day=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1))]
    for row in df_ss_day.itertuples(index=False):
        home_parts=row.homeTeamShortLow.split(' ')
        away_parts=row.awayTeamShortLow.split(' ')
        df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, row.awayTeamShortLow, type='00')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, row.awayTeamShortLow, type='10')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, row.awayTeamShortLow, type='01')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, row.awayTeamShortLow, type='11')
        if len(df_bf_slice.index)!=1 and ' ' in row.homeTeamShortLow:
            homeFirst=home_parts[0]
            homeLast=home_parts[-1]
            if len(homeFirst)>2:
                df_bf_slice=slice_df(df_bf_day, row.countryCode, homeFirst, row.awayTeamShortLow, type='10')
            if len(df_bf_slice.index)!=1 and len(homeLast)>2:
                df_bf_slice=slice_df(df_bf_day, row.countryCode, homeLast, row.awayTeamShortLow, type='10')
        if len(df_bf_slice.index)!=1 and ' ' in row.awayTeamShortLow:
            awayFirst=away_parts[0]
            awayLast=away_parts[-1]
            if len(df_bf_slice.index)!=1 and len(awayFirst)>2:
                df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, awayFirst, type='01')
            if len(df_bf_slice.index)!=1 and len(awayLast)>2:
                df_bf_slice=slice_df(df_bf_day, row.countryCode, row.homeTeamShortLow, awayLast, type='01')
            if len(df_bf_slice.index)!=1 and ' ' in row.homeTeamShortLow:
                homeFirst=home_parts[0]
                homeLast=home_parts[-1]
                if len(df_bf_slice.index)!=1 and len(homeFirst)>2 and len(awayFirst)>2:
                    df_bf_slice=slice_df(df_bf_day, row.countryCode, homeFirst, awayFirst, type='11')
                if len(df_bf_slice.index)!=1 and len(homeFirst)>2 and len(awayLast)>2:
                    df_bf_slice=slice_df(df_bf_day, row.countryCode, homeFirst, awayLast, type='11')
                if len(df_bf_slice.index)!=1 and len(awayLast)>2 and len(awayFirst)>2:
                    df_bf_slice=slice_df(df_bf_day, row.countryCode, homeLast, awayFirst, type='11')
                if len(df_bf_slice.index)!=1 and len(homeLast)>2 and len(awayLast)>2:
                    df_bf_slice=slice_df(df_bf_day, row.countryCode, homeLast, awayLast, type='11')
        if len(df_bf_slice.index)==1:
            eventId,home_id,away_id,bf_ht,bf_at=df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]
            print(f'found: {row.homeTeamShort}/{bf_ht} - {row.awayTeamShort}/{bf_at}')
            binds.append({
                'bf_home_name':bf_ht,
                'bf_away_name':bf_at,
                'bf_home_id':home_id,
                'bf_away_id':away_id,
                'bf_eventId':eventId,
                'ss_home':row.homeTeamShort,
                'ss_away':row.awayTeamShort,
                'ss_id':row.id
            })
    #
    d+=timedelta(days=1)
    #break
pd.DataFrame(binds).to_csv('data/binds.csv', index=False)

/Mirandes - Ponferradina/Ponferradina
found: Reggina /Reggina - Pescara/Pescara
found: SPAL/Spal - Cosenza/Cosenza
found: Venezia/Venezia - Frosinone/Frosinone
found: Entella/Entella - Reggiana/Reggiana
found: Vicenza/LR Vicenza Virtus - Pordenone/Pordenone
found: Auxerre/Auxerre - AC Ajaccio/AC Ajaccio
found: Clermont/Clermont - Rodez/Rodez
found: Dunkerque/Dunkerque - Guingamp/Guingamp
found: Admira/Admira Wacker - Ried/SV Ried
found: La Serena/La Serena - Unión Española/Union Espanola
found: Sarpsborg 08/Sarpsborg - Stabæk/Stabaek
found: Montana/Montana - Slavia/Slavia Sofia
found: Šibenik/Sibenik - Osijek/Osijek
found: Gorica/HNK Gorica - Hajduk/Hajduk Split
found: Opava/SFC Opava - Teplice/Teplice
found: Cova da Piedade/Cova da Piedade - Arouca/Arouca
found: Domžale/Domzale - Gorica/Gorica
found: Inhulets/Inhulets Petrove - FC Minaj/FC Minaj
2020-10-04 00:00:00
found: Southampton/Southampton - West Brom/West Brom
found: Man Utd/Man Utd - Tottenham/Tottenham
found: Osasuna/Osasuna 

## Teams
Extract teams with Unicode 

In [101]:
df_teams=pd.read_csv('data/teams_names.csv', index_col=None)
df_a=df_ss[['homeTeam','homeTeamShort','country','tournament']]
df_a.columns=['Team','TeamShort','country','tournament']
df_b=df_ss[['awayTeam','awayTeamShort','country','tournament']]
df_b.columns=['Team','TeamShort','country','tournament']
df_teams_new=pd.concat([df_a,df_b], axis=0).drop_duplicates()
df_teams_new['TeamEn']=df_teams_new['Team'].replace(dicUnicode2En, regex=True)
df_teams_new['TeamShortEn']=df_teams_new['TeamShort'].replace(dicUnicode2En, regex=True)
df_teams_new=df_teams_new.loc[df_teams_new['Team'].str.contains(r'[^\x00-\x7F]+')]
#df_teams_new['TeamEn']=''
#df_teams_new['TeamShortEn']=''

df_teams=pd.concat([df_teams,df_teams_new], axis=0).drop_duplicates(subset=['Team','country','tournament'], keep='first')

#df_teams_new.to_csv('data/teams_names1.csv', index=False)
df_teams.to_csv('data/teams_names.csv', index=False)

In [102]:
df_teams

,Team,TeamShort,country,tournament,TeamEn,TeamShortEn
0,Málaga,Málaga,spain,laliga,Malaga,Malaga
1,Académica Coimbra,Académica,portugal,primeira-liga,Academica Coimbra,Academica
2,Club América,América,mexico,liga-mx-clausura,Club America,America
3,Almería,Almería,spain,laliga,Almeria,Almeria
4,Nîmes Olympique,Nîmes,france,ligue-2,Nimes Olympique,Nimes
...,...,...,...,...,...,...
881,Colón de Santa Fe,Colón,argentina,copa-de-la-liga-profesional-group-2,NaN,NaN
882,Nõmme Kalju,Kalju,estonia,premium-liiga,NaN,NaN
883,Umeå,Umeå,sweden,superettan,NaN,NaN
884,Lillestrøm SK,Lillestrøm,norway,1st-division,NaN,NaN


## Make binded SS list


In [5]:
ds = pytz.timezone(local_tz).localize(datetime(2015, 5, 1))
ss=df_ss.loc[df_ss['ts']>=ds]

df_binds=pd.read_csv('data/binds.csv', index_col=None)
df_binds = df_binds.loc[~((df_binds['bf_home_name'].str.contains('\(Y\)')) | (df_binds['bf_home_name'].str.contains(' U20')) | (df_binds['bf_away_name'].str.contains('\(Y\)')) | (df_binds['bf_away_name'].str.contains(' U20'))) ]

df_ss_binded=ss.merge(df_binds, left_on=['id','homeTeamShort', 'awayTeamShort'] , right_on=['ss_id','ss_home','ss_away'], how='inner')
df_ss_binded=df_ss_binded.drop_duplicates()
df_semi = ss.loc[~(ss['id'].isin(df_ss_binded['id']))]

df_semi=df_semi.merge(df_binds[['bf_home_name', 'bf_home_id', 'ss_home']].drop_duplicates(), left_on=['homeTeamShort'] , right_on=['ss_home'], how='left')
df_semi=df_semi.merge(df_binds[['bf_away_name', 'bf_away_id', 'ss_away']].drop_duplicates(), left_on=['awayTeamShort'] , right_on=['ss_away'], how='left')
df_not=df_semi.loc[(df_semi['bf_home_id'].isna()) & (df_semi['bf_away_id'].isna())]
df_semi = df_semi.loc[~(df_semi['id'].isin(df_not['id']))]
df_semi=df_semi.drop_duplicates()
print(' full: {}, semi: {}, non: {}'.format(len(df_ss_binded.index),len(df_semi.index),len(df_not.index)) )

 full: 52256, semi: 46435, non: 2402


In [6]:
binds=[]
binded_total=0
for row in df_semi.itertuples(index=False):
    d=row.ts.replace(tzinfo=None)
    if row.bf_home_id>0:
        #print(0,row.bf_home_id)
        df_bf_slice=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1)) & (df_bf['home_id']==row.bf_home_id)]
        if len(df_bf_slice.index)==1:
            ssid, sshid, sshname, ssaname = (row.id,row.bf_home_id,row.homeTeamShort,row.awayTeamShort)
            bfid, bfhid, bfaid, bfhname, bfaname = (df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]).values
            print('Found on home id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(sshid, sshname,bfhname,ssaname,bfaname,ssid,bfid ))
            binds.append({
                'bf_home_name':bfhname,
                'bf_away_name':bfaname,
                'bf_home_id':bfhid,
                'bf_away_id':bfaid,
                'bf_eventId':bfid,
                'ss_home':sshname,
                'ss_away':ssaname,
                'ss_id':ssid
            })
            binded_total+=1
    else:
        #print(1,row.bf_away_id)
        df_bf_slice=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1)) & (df_bf['away_id']==row.bf_away_id)]
        if len(df_bf_slice.index)==1:
            ssid, ssaid, sshname, ssaname = (row.id,row.bf_away_id,row.homeTeamShort,row.awayTeamShort)
            bfid, bfhid, bfaid, bfhname, bfaname = (df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]).values
            print('Found on away id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(ssaid, sshname,bfhname,ssaname,bfaname,ssid,bfid ))
            binds.append({
                'bf_home_name':bfhname,
                'bf_away_name':bfaname,
                'bf_home_id':bfhid,
                'bf_away_id':bfaid,
                'bf_eventId':bfid,
                'ss_home':sshname,
                'ss_away':ssaname,
                'ss_id':ssid
            })
            binded_total+=1
print (f'Done! {binded_total} matches binded')
pd.concat([df_binds,pd.DataFrame(binds)], axis=0).drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/binds.csv', index=False)
    


Done! 0 matches binded


In [12]:
df_binds.drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/binds.csv', index=False)

In [7]:
df_ss_binded.drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/matches_binded.csv', index=False)

In [16]:
df_ss

,awayTeam,homeScoreHT,homeTeam,ts,awayScoreFT,status,winnerCode,country,done,awayTeamShort,...,tournament,round,coverage,awayScoreHT,homeScoreFT,startTimestamp,awayScoreET,countryCode,homeTeamShortLow,awayTeamShortLow
0,Derby County,0.0,Ipswich Town,2015-01-10 12:15:00+00:00,1.0,100,2,england,1,Derby,...,championship,25.0,1,0.0,0.0,1420892100,1,GB,ipswich,derby
1,Stade Brestois 29,1.0,Troyes,2015-01-10 13:00:00+00:00,0.0,100,1,france,1,Brest,...,ligue-2,19.0,1,0.0,1.0,1420894800,0,FR,troyes,brest
2,AEL Kalloni,1.0,Veria NFC,2015-01-10 13:00:00+00:00,1.0,100,3,greece,1,Kalloni,...,super-league,11.0,1,1.0,1.0,1420894800,1,GR,veria,kalloni
3,Queens Park Rangers,2.0,Burnley,2015-01-10 15:00:00+00:00,1.0,100,1,england,1,QPR,...,premier-league,21.0,1,1.0,2.0,1420902000,1,GB,burnley,qpr
4,Newcastle United,1.0,Chelsea,2015-01-10 15:00:00+00:00,0.0,100,1,england,1,Newcastle,...,premier-league,21.0,1,0.0,2.0,1420902000,0,GB,chelsea,newcastle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53991,Colorado Rapids,1.0,Chicago Fire,2019-04-20 17:00:00+00:00,1.0,100,1,usa,1,Colorado,...,major-league-soccer,NaN,1,1.0,4.0,1555779600,1,US,chicago fire,colorado
53992,Impact Montreal,2.0,Philadelphia Union,2019-04-20 17:00:00+00:00,0.0,100,1,usa,1,Montreal,...,major-league-soccer,NaN,1,0.0,3.0,1555779600,0,US,philadelphia,montreal
53993,FC Luzern,0.0,FC St. Gallen 1879,2019-04-20 17:00:00+00:00,2.0,100,2,switzerland,1,Luzern,...,super-league,30.0,1,1.0,1.0,1555779600,2,CH,st. gallen,luzern
53994,FC Thun,0.0,Grasshopper Club Zürich,2019-04-20 17:00:00+00:00,1.0,100,3,switzerland,1,Thun,...,super-league,30.0,1,0.0,1.0,1555779600,1,CH,grasshoppers,thun


In [118]:
df=pd.concat([df1,df2], axis=0)

In [56]:
df=pd.read_csv('data/matches_2015.csv', index_col=None)

In [35]:
df.to_csv('data/matches_2015.csv', index=False)

In [144]:
dfs=[]
for y in range(2015,2021):
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_matches.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
df=pd.concat(dfs, axis=0)
df=df.loc[~df['home_id'].isna()]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','draw_id','draw_name','inplayTime','halfTime']]
df=df[df['eventName'].str.contains(' v ')]
df[['ht','at']]=df.apply(lambda x: x.eventName.split(' v '), axis=1, result_type="expand")
df=df.reset_index(drop=True)
df1=df.loc[(df['draw_name']!='The Draw')]
df2=df.loc[(df['draw_name']=='The Draw')]
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['home_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['away_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df=pd.concat([df1,df2], axis=0)
df.dropna(subset=['inplayTime'], inplace=True)
df=df[~((df['home_name']=='test1') | (df['away_name']=='test1') | (df['home_name']=='test2') | (df['away_name']=='test2'))]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','inplayTime','halfTime']]
df.to_csv('data/bf_matches.csv', index=False)

data/bf\2018-Oct-16_matches.csv
data/bf\2018-Oct-17_matches.csv
data/bf\2018-Oct-18_matches.csv
data/bf\2018-Oct-19_matches.csv
data/bf\2018-Oct-20_matches.csv
data/bf\2018-Oct-21_matches.csv
data/bf\2018-Oct-22_matches.csv
data/bf\2018-Oct-23_matches.csv
data/bf\2018-Oct-24_matches.csv
data/bf\2018-Oct-25_matches.csv
data/bf\2018-Oct-26_matches.csv
data/bf\2018-Oct-27_matches.csv
data/bf\2018-Oct-28_matches.csv
data/bf\2018-Oct-29_matches.csv
data/bf\2018-Oct-30_matches.csv
data/bf\2018-Oct-31_matches.csv
data/bf\2018-Sep-01_matches.csv
data/bf\2018-Sep-02_matches.csv
data/bf\2018-Sep-03_matches.csv
data/bf\2018-Sep-04_matches.csv
data/bf\2018-Sep-05_matches.csv
data/bf\2018-Sep-06_matches.csv
data/bf\2018-Sep-07_matches.csv
data/bf\2018-Sep-08_matches.csv
data/bf\2018-Sep-09_matches.csv
data/bf\2018-Sep-10_matches.csv
data/bf\2018-Sep-11_matches.csv
data/bf\2018-Sep-12_matches.csv
data/bf\2018-Sep-13_matches.csv
data/bf\2018-Sep-14_matches.csv
data/bf\2018-Sep-15_matches.csv
data/bf\

In [177]:
df.to_csv('data/bf_matches.csv', index=False)

In [204]:
cols=['eventId','clk','ltp','id','ip']
cols_noid=['eventId','clk','ltp','ip']
df_matches=pd.read_csv('data/bf/bf_matches.csv', index_col=None)
for y in range(2015,2021):
    dfs=[]
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_odds.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
    df=pd.concat(dfs, axis=0)
    df=df[df['eventId'].isin(df_matches['eventId'])]
    df['ip']=0
    df.loc[df['inplay'],'ip']=1
    df=df[cols]
    df_un=df.loc[df['id']==ID_UNDER][cols_noid]
    df_ov=df.loc[df['id']==ID_OVER][cols_noid]
    df_draw=df.loc[df['id']==ID_DRAW][cols_noid]
    df_home=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','home_id'])[cols_noid]
    df_away=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','away_id'])[cols_noid]
    #df_12=df.loc[(df['id']!=ID_UNDER) & (df['id']!=ID_OVER) & (df['id']!=ID_DRAW)]
    df_un.to_csv(f'data/bf_un_{y}.csv', index=False)
    df_ov.to_csv(f'data/bf_ov_{y}.csv', index=False)
    df_draw.to_csv(f'data/bf_draw_{y}.csv', index=False)
    df_home.to_csv(f'data/bf_home_{y}.csv', index=False)
    df_away.to_csv(f'data/bf_away_{y}.csv', index=False)
    #break

dds.csv
data/bf/days\2018-Sep-23_odds.csv
data/bf/days\2018-Sep-24_odds.csv
data/bf/days\2018-Sep-25_odds.csv
data/bf/days\2018-Sep-26_odds.csv
data/bf/days\2018-Sep-27_odds.csv
data/bf/days\2018-Sep-28_odds.csv
data/bf/days\2018-Sep-29_odds.csv
data/bf/days\2018-Sep-30_odds.csv
data/bf/days\2019-Apr-01_odds.csv
data/bf/days\2019-Apr-02_odds.csv
data/bf/days\2019-Apr-03_odds.csv
data/bf/days\2019-Apr-04_odds.csv
data/bf/days\2019-Apr-05_odds.csv
data/bf/days\2019-Apr-06_odds.csv
data/bf/days\2019-Apr-07_odds.csv
data/bf/days\2019-Apr-08_odds.csv
data/bf/days\2019-Apr-09_odds.csv
data/bf/days\2019-Apr-10_odds.csv
data/bf/days\2019-Apr-11_odds.csv
data/bf/days\2019-Apr-12_odds.csv
data/bf/days\2019-Apr-13_odds.csv
data/bf/days\2019-Apr-14_odds.csv
data/bf/days\2019-Apr-15_odds.csv
data/bf/days\2019-Apr-16_odds.csv
data/bf/days\2019-Apr-17_odds.csv
data/bf/days\2019-Apr-18_odds.csv
data/bf/days\2019-Apr-19_odds.csv
data/bf/days\2019-Apr-20_odds.csv
data/bf/days\2019-Apr-21_odds.csv
data/b

In [181]:
df_matches=pd.read_csv('data/bf_matches.csv', index_col=None)

